In [2]:
import pandas as pd
import pickle
import numpy as npy
dict_table = pd.read_csv('2022_world_cup_groups.csv')
df_fixture = pd.read_csv('2022_world_cup_matches.csv')
df3 = pd.read_csv('data_dictionary.csv')
df_historical_data = pd.read_csv('international_matches.csv')
df4 = pd.read_csv('world_cup_matches.csv')
df5 = pd.read_csv('world_cups.csv')

#CALCULATING TEAM STRENGTH

In [3]:
df_home = df_historical_data[['Home Team', 'Home Goals', 'Away Goals']]
df_away = df_historical_data[['Away Team', 'Home Goals', 'Away Goals']]

In [4]:
df_home = df_home.rename(columns={'Home Team': 'Team', 'Home Goals': 'GoalsScored', 'Away Goals': 'GoalsConceded'})
df_away = df_away.rename(columns={'Away Team': 'Team', 'Home Goals': 'GoalsConceded', 'Away Goals': 'GoalsScored'})

In [5]:
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

FUNCTION PREDICT POINTS

In [6]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

PREDICT WORLD CUP
group stage

In [7]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [8]:
Group_A = dict_table.loc[0:3, ['Group','Team']]
Group_B = dict_table.loc[4:7, ['Group','Team']]
Group_C = dict_table.loc[8:11, ['Group','Team']]
Group_D = dict_table.loc[12:15, ['Group','Team']]
Group_E = dict_table.loc[16:19, ['Group','Team']]
Group_F = dict_table.loc[20:23, ['Group','Team']]
Group_G = dict_table.loc[24:27, ['Group','Team']]
Group_H = dict_table.loc[28:31, ['Group','Team']]

group = [Group_A,Group_B,Group_C,Group_D,Group_E,Group_F,Group_G,Group_H]

In [9]:
# run all matches in the group stage and update group tables
"""for group in dict_table:
    teams_in_group = dict_table['Team'].values
    df_fixture_group_6 = df_fixture_group_48
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)
"""
  

"for group in dict_table:\n    teams_in_group = dict_table['Team'].values\n    df_fixture_group_6 = df_fixture_group_48\n    for index, row in df_fixture_group_6.iterrows():\n        home, away = row['home'], row['away']\n        points_home, points_away = predict_points(home, away)\n        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home\n        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away\n    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()\n    dict_table[group] = dict_table[group][['Team', 'Pts']]\n    dict_table[group] = dict_table[group].round(0)\n"